In [2]:
import pandas as pd # 0.23.0
import numpy as np

In [24]:
fileName = 'combinations.csv'
df = pd.read_csv(fileName)
df

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,templates/Temp_NoHybrid_Bifuel.xml,BFE01emptyB1040$$,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
1,templates/Temp_OVC_Bifuel.xml,BFE01vchevB10400$,N1,OVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
2,templates/Temp_NOVC_Bifuel.xml,BFE01nchevB10400$,N1,NOVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
3,templates/Temp_NoHybrid_Bifuel.xml,BFE38emptyB1040$$,N1,Empty,B,10,40,-,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
4,templates/Temp_OVC_Bifuel.xml,BFE38vchevB10400$,N1,OVC-HEV,B,10,40,0,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
...,...,...,...,...,...,...,...,...,...,...,...
58,templates/Temp_NOVC_Monofuel.xml,BFE01nchevM20$$0$,N1,NOVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"
59,templates/Temp_OVC_Monofuel.xml,BFE38vchevM20$$0$,N1,OVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
60,templates/Temp_NOVC_Monofuel.xml,BFE38nchevM20$$0$,N1,NOVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
61,templates/Temp_OVC_Monofuel.xml,BFE30vchevM20$$0$,M1,OVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"


In [25]:
cols = ['#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
        '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3',
        'FuelCode4', 'Info manueller Change in DB: vehicleNatureCode ']

allLists = []

for col in cols:
  allLists.append(df[col].unique())
  print(df[col].unique())

len(allLists)
allLists[7] = ['-', '35', '38']

['N1' 'M1']
['Empty' 'OVC-HEV' 'NOVC-HEV' 'OVC-FCHV' 'NOVC-FCHV']
['B' 'T' 'M' '-' 'D']
[10 50  0 40 20]
['40' '-']
['-' '0' '44']
['-' '0']
['-' '38']


In [26]:
newDF = pd.DataFrame(columns=cols)

for a in allLists[0]:
  for b in allLists[1]:
    for c in allLists[2]:
      for d in allLists[3]:
        for e in allLists[4]:
          for f in allLists[5]:
            for g in allLists[6]:
              for h in allLists[7]:

                car = [a, b, c, d, e, f, g, h]
                newDF.loc[len(newDF)] = car

---------------------------------------

In [27]:
newDF['#VIN'] = '-'
newDF = newDF[['#VIN', '#VehicleCategoryCode', 'ClassOfHybridVehicleCode',
               '#FuelTypeCode', '#FuelCode1', '#FuelCode2', '#FuelCode3',
               'FuelCode4', 'Info manueller Change in DB: vehicleNatureCode ']]

In [28]:
# map the following values
dict_VCC = {'N1': '01', 'M1': '30'} # 35 and 38 need to be mapped
dict_CHVC = {'Empty': 'empty', 'OVC-HEV': 'vchev',
             'NOVC-HEV': 'nchev', 'OVC-FCHV': 'cfchv',
             'NOVC-FCHV': 'nfchv'}


def createVIN(row):

  emptyElement = 'x'

  newVIN = 'BFE'

  # check for N1
  vcc = row['#VehicleCategoryCode']

  if vcc == 'N1':

    com = row['Info manueller Change in DB: vehicleNatureCode ']

    if com  != '-':
      newVIN = newVIN + com
    else:
      newVIN = newVIN + dict_VCC[vcc]
  else:
    newVIN = newVIN + dict_VCC[vcc]


  newVIN = newVIN + dict_CHVC[row['ClassOfHybridVehicleCode']]


  fuelTypeCode = row['#FuelTypeCode']

  if fuelTypeCode == '-':
    newVIN = newVIN + emptyElement
  else:
    newVIN = newVIN + fuelTypeCode

  for col in ['#FuelCode1','#FuelCode2', '#FuelCode3']:

    val = str(row[col])

    if len(val) < 2:
      if val == '-':
        val = emptyElement + emptyElement
      else:
        val = val + emptyElement

    newVIN = newVIN + val

  return newVIN

In [29]:
newDF['#VIN'] = newDF.apply(lambda row: createVIN(row), axis=1)


---------------------------------------

In [30]:
indx = newDF[(newDF['#VehicleCategoryCode']=='M1') & (newDF['Info manueller Change in DB: vehicleNatureCode ']!='-')].index
newDF.drop(indx, inplace=True)

In [31]:
print('N1: ', newDF[newDF['#VehicleCategoryCode'] == 'N1']['Info manueller Change in DB: vehicleNatureCode '].unique())
print('M1: ', newDF[newDF['#VehicleCategoryCode'] == 'M1']['Info manueller Change in DB: vehicleNatureCode '].unique())

N1:  ['-' '35' '38']
M1:  ['-']


In [32]:
len(newDF)

6000

In [33]:
len(newDF['#VIN'].unique())

3000

-------------------------------------------------------------


_Empty_ cannot have a fuelCode4 == 0

In [34]:
tmp = newDF[newDF['ClassOfHybridVehicleCode'] == 'Empty'].copy()

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']
  fuelCode3 = row['#FuelCode3']

  if fuelCode4 == '0':
    toDrop.append(index)
    continue

In [35]:
newDF.drop(toDrop, inplace=True)

In [36]:
len(newDF)

5400

In [37]:
len(newDF['#VIN'].unique())

3000

-----------------------------

_#FuelTypeCode_ == T (AND ClassOfHybridVehicleCode != _Empty_) has _0_ for _FuelCode4_

In [38]:
tmp = newDF[newDF['#FuelTypeCode'] == 'T'].copy()
tmp = tmp[tmp['ClassOfHybridVehicleCode'].isin(['NOVC-HEV', 'NOVC-FCHV', 'OVC-HEV', 'OVC-FCHV'])]

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']

  if fuelCode4 == '-':
    toDrop.append(index)
    continue

In [39]:
newDF.drop(toDrop, inplace=True)

In [40]:
len(newDF)

4920

In [41]:
len(newDF['#VIN'].unique())

3000

---------------------------------------------------------------------

*#FuelTypeCode* == M, _ClassOfHybridVehicle_ != _Empty_, _#FuelCode1 == 50 --> *FuelCode4* = 0

In [42]:
tmp = newDF[newDF['#FuelTypeCode'] == 'M'].copy()
tmp = tmp[tmp['ClassOfHybridVehicleCode'].isin(['NOVC-HEV', 'NOVC-FCHV', 'OVC-HEV', 'OVC-FCHV'])]

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']
  fuelCode1 = row['#FuelCode1']

  if fuelCode4 == '-' and fuelCode1 == 50:
    toDrop.append(index)
    continue

In [43]:
newDF.drop(toDrop, inplace=True)

In [44]:
len(newDF)

4824

In [45]:
len(newDF['#VIN'].unique())

3000

- - - - - - - - - - - - - - - - - - - - - - - - - - - -:

In [49]:
tmp = newDF[newDF['#FuelTypeCode'] == 'M'].copy()
tmp = tmp[tmp['ClassOfHybridVehicleCode'].isin(['NOVC-HEV', 'NOVC-FCHV', 'OVC-HEV', 'OVC-FCHV'])]

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']
  fuelCode1 = row['#FuelCode1']

  if fuelCode4 == '0' and fuelCode1 != 50:
    toDrop.append(index)
    continue

In [50]:
newDF.drop(toDrop, inplace=True)
print(len(newDF))
print(len(newDF['#VIN'].unique()))

3480
3000


--------------------------------------------------

_#FuelTypeCode_ == *B* || *D*, *ClassOfHybridVehicle* != *Empty* --> _FuelCode4_ = -

In [46]:
tmp = newDF[newDF['#FuelTypeCode'].isin(['B', 'D'])].copy()
tmp = tmp[tmp['ClassOfHybridVehicleCode'].isin(['NOVC-HEV', 'NOVC-FCHV', 'OVC-HEV', 'OVC-FCHV'])]

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']

  if fuelCode4 == '0':
    toDrop.append(index)
    continue

In [47]:
newDF.drop(toDrop, inplace=True)
print(len(newDF))
print(len(newDF['#VIN'].unique()))

3864
3000


------------------------------------------------------------

_#FuelTypeCode_ == *-*, *ClassOfHybridVehicle* != *Empty* --> _FuelCode4_ = -

In [70]:
tmp = newDF[newDF['#FuelTypeCode'] == '-'].copy()
tmp = tmp[tmp['ClassOfHybridVehicleCode'].isin(['NOVC-HEV', 'NOVC-FCHV', 'OVC-HEV', 'OVC-FCHV'])]

toDrop = []

for index, row in tmp.iterrows():

  fuelCode4 = row['FuelCode4']

  if fuelCode4 == '0':
    toDrop.append(index)
    continue

In [71]:
newDF.drop(toDrop, inplace=True)
print(len(newDF))
print(len(newDF['#VIN'].unique()))

3000
3000


-----------------------------------------------------

Save file

In [ ]:
newDF.to_csv('allPossibleCombinations.csv', index=False)